# Amazon Nova Act SDK를 활용한 라이브 뷰 브라우저 도구

## 개요

이 튜토리얼에서는 Nova Act SDK를 사용하여 Amazon Bedrock Agentcore 브라우저 도구와 상호작용하고 브라우저를 실시간으로 확인하는 방법을 알아봅니다.

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| 에이전트 프레임워크 | Nova Act |
| LLM 모델 | Amazon Nova Act 모델 |
| 튜토리얼 구성 요소 | NovaAct를 사용하여 브라우저 도구와 실시간으로 상호작용 |
| 튜토리얼 수직 | 교차 수직 |
| 예제 복잡성 | 쉬움 |
| 사용 SDK | Amazon BedrockAgentCore Python SDK, Nova Act |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Nova Act를 브라우저 도구와 함께 사용하고 브라우저를 실시간으로 확인하는 방법을 설명합니다.

이 예시에서는 Nova Act 에이전트에 자연어 명령을 전송하여 Bedrock Agentcore 브라우저에서 작업을 수행하고 브라우저를 실시간으로 확인하도록 하겠습니다.

<div style="text-align:left">
<img src="./images/browser-tool.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* 브라우저 도구 사용 및 실시간 확인
* 브라우저 도구와 함께 Nova Act 사용

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Nova Act SDK and API key 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 라이브 뷰가 포함된 Bedrock Agentcore 브라우저 도구와 NovaAct 함께 사용하기

여기서는 Amazon DCV SDK를 통해 Bedrock Agentcore 브라우저 도구에 연결하는 도우미 함수를 사용합니다.

In [ ]:
%%writefile live_view_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
from rich.panel import Panel
import sys
import json
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def live_view_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    """Run the browser live viewer with display sizing."""
    console.print(
        Panel(
            "[bold cyan]Browser Live Viewer[/bold cyan]\n\n"
            "This demonstrates:\n"
            "• Live browser viewing with DCV\n"
            "• Configurable display sizes (not limited to 900×800)\n"
            "• Proper display layout callbacks\n\n"
            "[yellow]Note: Requires Amazon DCV SDK files[/yellow]",
            title="Browser Live Viewer",
            border_style="blue",
        )
    )

    try:
        # Step 1: Create browser session
        with browser_session(region) as client:
            ws_url, headers = client.generate_ws_headers()

            # Step 2: Start viewer server
            console.print("\n[cyan]Step 3: Starting viewer server...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            # Step 3: Show features
            console.print("\n[bold green]Viewer Features:[/bold green]")
            console.print(
                "• Default display: 1600×900 (configured via displayLayout callback)"
            )
            console.print("• Size options: 720p, 900p, 1080p, 1440p")
            console.print("• Real-time display updates")
            console.print("• Take/Release control functionality")

            console.print("\n[yellow]Press Ctrl+C to stop[/yellow]")

            # Step 4: Use Nova Act to interact with the browser
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                result = nova_act.act(prompt)
                console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    result = live_view_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )

    with open('result.txt', 'w') as f:
        f.write(str(result))

    console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")

#### Running the script
Paste your Nova Act API key below before running the script. 

In [ ]:
NOVA_ACT_KEY= ''  ### Paste your Nova Act Key here

In [ ]:
!python live_view_with_nova_act.py --prompt "Search for macboks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

### 백그라운드에서 무슨 일이 일어났을까요?
* 브라우저 클라이언트를 인스턴스화하고 세션을 시작했습니다.
* 그런 다음 `BrowserViewerServer`를 사용하여 브라우저 세션에 연결하여 로컬에서 세션을 확인했습니다.
* 그런 다음 Nova Act 에이전트를 생성하고 브라우저 세션의 세부 정보를 전달했습니다.
* 그런 다음 Nova Act 에이전트에 자연어 명령을 전송하여 실시간으로 동작을 확인했습니다.

## 브라우저에서 CAPTCHA 처리하기
다음으로, 브라우저에서 CAPTCHA를 처리하는 방법을 살펴보겠습니다. CAPTCHA의 목적은 봇이 아닌 사람이 웹사이트와 상호 작용하도록 하는 것입니다. 따라서 에이전트가 CAPTCHA를 해결하도록 허용하지 않고, 에이전트가 CAPTCHA를 해결하고 에이전트가 계속 진행하도록 합니다.

새 스크립트를 만들어 보겠습니다. Nova Act를 사용하면 페이지에 캡차가 있는지 확인할 수 있습니다. 이 기능을 사용하여 Nova Act를 실행하기 전에 캡차를 처리하겠습니다.

#### 스크립트가 실행되면 브라우저의 로컬 뷰가 표시됩니다. 스크립트 실행 중에 캡차가 표시되면 캡차를 직접 해결하세요. 스크립트는 캡차가 해결될 때까지 대기합니다.

##### 참고: 캡차가 표시되지 않고 스크립트가 완전히 실행되면 캡차가 나타날 때까지 스크립트를 다시 실행해 보세요.

In [ ]:
%%writefile captcha_with_nova_act.py
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct, BOOL_SCHEMA, ActAgentError
from rich.console import Console
from rich.panel import Panel
import sys
import json
import time
import argparse
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer


console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def contains_human_validation_error(err):
    """
    Recursively check if the error or its message attribute indicates HumanValidationError.
    """
    if err is None:
        return False

    # Direct string check
    if isinstance(err, str) and "HumanValidationError" in err:
        return True

    # If err has 'message' attribute that's string or another error, recurse
    if hasattr(err, "message"):
        return contains_human_validation_error(err.message)

    # If err has string representation containing the error text
    if "HumanValidationError" in str(err):
        return True

    return False

def live_view_with_nova_act(steps, starting_page, nova_act_key, region="us-west-2"):
    """Run the browser live viewer with display sizing."""
    console.print(
        Panel(
            "[bold cyan]Browser Live Viewer[/bold cyan]\n\n"
            "This demonstrates:\n"
            "• Live browser viewing with DCV\n"
            "• Configurable display sizes (not limited to 900×800)\n"
            "• Proper display layout callbacks\n\n"
            "[yellow]Note: Requires Amazon DCV SDK files[/yellow]",
            title="Browser Live Viewer",
            border_style="blue",
        )
    )
    result = None

    try:
        # Step 1: Create browser session
        with browser_session(region) as client:
            ws_url, headers = client.generate_ws_headers()

            # Step 2: Start viewer server
            console.print("\n[cyan]Step 3: Starting viewer server...[/cyan]")
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)

            # Step 3: Show features
            console.print("\n[bold green]Viewer Features:[/bold green]")
            console.print(
                "• Default display: 1600×900 (configured via displayLayout callback)"
            )
            console.print("• Size options: 720p, 900p, 1080p, 1440p")
            console.print("• Real-time display updates")
            console.print("• Take/Release control functionality")

            console.print("\n[yellow]Press Ctrl+C to stop[/yellow]")

            # Step 4: Use Nova Act to interact with the browser
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                
                for step_index, step in enumerate(steps):
                    max_retries = 3
                    retry_count = 0
                    
                    while retry_count < max_retries:
                        try:
                            print(f"Executing step {step_index + 1}/{len(steps)}: {step}")
                            result = nova_act.act(step)
                            console.print(f"\n[bold green]Step {step_index + 1} Result:[/bold green] {result}")
                            break  # Success, move to next step
                            
                        except ActAgentError as err:
                            # Check for human validation in the error message or structure
                            if contains_human_validation_error(err):
                                print("CAPTCHA detected! Please solve it in the browser.")
                                captcha_wait_attempts = 0
                                max_captcha_wait_attempts = 8
                                
                                while captcha_wait_attempts < max_captcha_wait_attempts:
                                    try:
                                        time.sleep(10)  # Give user time to solve captcha
                                        captcha_result = nova_act.act(
                                            "Is there a captcha on the screen?", schema=BOOL_SCHEMA
                                        )
                                        
                                        if captcha_result.matches_schema and not captcha_result.parsed_response:
                                            print("Captcha solved, continuing with current step...")
                                            # Don't increment retry_count so we retry the current step without penalty
                                            break
                                        else:
                                            print(f"Captcha still present. Waiting... (Attempt {captcha_wait_attempts + 1}/{max_captcha_wait_attempts})")
                                            captcha_wait_attempts += 1
                                            
                                    except Exception as captcha_check_err:
                                        print(f"Error checking captcha status: {str(captcha_check_err)}")
                                        captcha_wait_attempts += 1
                                        time.sleep(5)
                                
                                if captcha_wait_attempts >= max_captcha_wait_attempts:
                                    print("Maximum captcha wait attempts reached. Trying to continue anyway.")
                                    retry_count += 1
                                
                            else:
                                print(f"Non-captcha error occurred: {str(err)}")
                                retry_count += 1
                                time.sleep(5)
                                
                        except Exception as general_err:
                            print(f"Unexpected error on step {step_index + 1}: {str(general_err)}")
                            retry_count += 1
                            time.sleep(5)
                            
                    if retry_count >= max_retries:
                        console.print(f"\n[bold red]Failed to complete step {step_index + 1} after {max_retries} attempts.[/bold red]")
                        if step_index < len(steps) - 1:
                            console.print("[yellow]Attempting to continue with next step...[/yellow]")
                
                # Final summary
                console.print("\n[bold blue]Task Execution Complete[/bold blue]")
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")
    return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--steps", required=False, help="JSON array or comma-separated list of steps to execute", 
                        default='["Search for AI news and press enter. If there is alredy AI news typed in search bar, then do not do anything", "Get the first AI news result, open the page and extract the title. Instead, if you see an AI summary, extract the first paragrpah of the summary and return"]')
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    # Parse steps - accept either a JSON array or comma-separated values
    try:
        # Try parsing as JSON first
        steps = json.loads(args.steps)
    except json.JSONDecodeError:
        # If not valid JSON, treat as comma-separated string
        steps = [step.strip() for step in args.steps.split(',')]

    # Ensure steps is a list
    if not isinstance(steps, list):
        steps = [steps]

    result = live_view_with_nova_act(
        steps, args.starting_page, args.nova_act_key, args.region
    )

In [ ]:
!python captcha_with_nova_act.py  --starting-page "https://www.google.com/" --nova-act-key {NOVA_ACT_KEY}

# Congratulations!